In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import nbfigtulz as ftl
import numpy as np

!mkdir -p img
ftl.config['img_dir'] = 'img'

In [2]:
def eval_pot(x, *, j, threshold=5):
    y = 1. / (2. * np.pi * j + x) + 1. / (2. * np.pi * (j + 1) - x)
    
    xs = []
    ys = []
    
    i_first = 0
    seq_done = False
    for i in range(len(x)):
        if abs(y[i]) > threshold:
            if not seq_done:
                seq_done = True
                xs.append(x[i_first:i])
                ys.append(y[i_first:i])
        elif seq_done:
            seq_done = False
            i_first = i
            
    xs.append(x[i_first:])
    ys.append(y[i_first:])
    
    return xs, ys


@ftl.with_context
def make_fig(n):
    x = np.linspace(-14, 20.5, 800)
    seqs = [eval_pot(x, j=j) for j in range(n)]
    
    fig, ax = plt.subplots()
    
    xticks = np.array([-4, -2, 0, 2, 4, 6]) * np.pi
    xticks_labels = [f'${x}$' for x in [r'-4\pi', r'-2\pi', r'0', r'2\pi', r'4\pi', r'6\pi']]
    ax.set_xticks(ticks=xticks, labels=xticks_labels)
    
    ax.set_xlabel(r'$\sigma_i$')
    ax.set_ylabel(r'$V_{\!3\mathrm{D}}^{(j)}(\sigma_i)$')
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(-4, 4)
    ax.grid()
    
    for j, seq in enumerate(seqs):
        label = f'$j={j}$'
        
        for x0, y0 in zip(*seq):
            s1 = (x0 > 0) 
            s2 = (x0 < 2. * np.pi) 
            ax.plot(x0[s1 & s2], y0[s1 & s2], color=f'C{j}', label=label)
            
            sel = (x0 < 0)
            ax.plot(x0[~s1], y0[~s1], '--', color=f'C{j}')
            
            sel = (x0 > 2. * np.pi)
            ax.plot(x0[~s2], y0[~s2], '--', color=f'C{j}')
            label = None
            
    ax.legend()
    
    return ftl.save_fig(fig, 'pot3D')


make_fig(3)

pot3D.png

In [3]:
def nu2D(x):
    return -1. / (1. - np.cos(x))


def nu3D(x, *, n):
    arg = x - np.pi
    
    y = 0
    for j in range(n):
        k = 2. * j + 1.
        y += k / ((k * np.pi) ** 2 -  arg ** 2) ** 2
            
    return -y / np.sinc(arg / np.pi)

In [4]:
@ftl.with_context
def make_fig():
    x = np.linspace(0., np.pi, 200)[10:]
    
    fig, ax = plt.subplots()
    ax.set_xlim(x[0], x[-1])
    ax2 = ax.twinx()
        
    xticks = np.array([.25, .5, .75, 1.]) * np.pi
    xticks_labels = [f'${x}\,\pi$' for x in ['0.25', '0.5', '0.75', '']]
    ax.set_xticks(ticks=xticks, labels=xticks_labels)
    
    ax.set_xlabel(r'$\sigma_i$')
    ax.set_ylabel(r'$\mathcal{V}_{2\mathrm{D}}(\sigma_i)$')
    ax2.set_ylabel(r'$\mathcal{V}_{3\mathrm{D}}(\sigma_i)$')
        
    ax.plot(x, nu2D(x), label=r'$d=2$', alpha=.8)
    ax2.plot(x, nu3D(x, n=100), color='C1', label=r'$d=3$', alpha=.8)
    
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    
    ax.legend([
        mpl.lines.Line2D([0], [0], color='C0'),
        mpl.lines.Line2D([0], [0], color='C1'),
    ], [
        r'$\mathcal{V}_{2\mathrm{D}}(\sigma_i)$',
        r'$\mathcal{V}_{3\mathrm{D}}(\sigma_i)$'
    ], loc='lower right')
        
    return ftl.save_fig(fig, 'fd')


make_fig()

fd.png

In [5]:
@ftl.with_context
def make_fig():
    x = np.linspace(0., np.pi, 200)[10:]
    
    fig, ax = plt.subplots()
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(1., 1.055)
    
    xticks = np.array([.25, .5, .75, 1.]) * np.pi
    xticks_labels = [f'${x}\,\pi$' for x in ['0.25', '0.5', '0.75', '']]
    ax.set_xticks(ticks=xticks, labels=xticks_labels)
    
    ax.set_xlabel(r'$\sigma_i$')
    ax.set_ylabel(r'$\left. \mathcal{V}^{(J)}_{3\mathrm{D}}(\sigma_i) \;\middle/\; \mathcal{V}^{(0)}_{3\mathrm{D}}(\sigma_i) \right.$')
        
    y0 = nu3D(x, n=1)
    
    for n in [2, 3, 8, 101]:
        y = nu3D(x, n=n) / y0
        ax.plot(x, y, label=f'$J={n-1}$', zorder=-n, alpha=.8)
        
    ax.legend()
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
        
    return ftl.save_fig(fig, 'f3D_ratio')


make_fig()

f3D_ratio.png